In [25]:
from llama_index.readers.file import  PandasCSVReader
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent
from dotenv import load_dotenv
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core import Settings
import os
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.tools import QueryEngineTool
from llama_index.core.workflow import Context

In [59]:
load_dotenv()
gemini_token = os.getenv("GOOGLE_API_KEY")
llm = GoogleGenAI(model="gemini-2.5-pro-exp-03-25", token=gemini_token)
embed_model = GoogleGenAIEmbedding(model="gemini-embedding-exp-03-07", token=gemini_token)


In [60]:
Settings.llm= llm
Settings.embed_model = embed_model

try:

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/ciuu"
    )
    ciuu_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [62]:
parser = PandasCSVReader(pandas_config={"encoding": "latin1"})
file_extractor = {".csv": parser} 
if not index_loaded:
    ciuu_docs = SimpleDirectoryReader(
        "./data", file_extractor=file_extractor
    ).load_data()

    ciuu_index = VectorStoreIndex.from_documents(ciuu_docs)

    ciuu_index.storage_context.persist(persist_dir="./storage/ciuu")


In [63]:
ciuu_engine = ciuu_index.as_query_engine(similarity_top_k=3)

In [64]:
query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=ciuu_engine,
        name="ciuu",
        description="use this tool to get information from the ciuu database",
    )
]

In [71]:
ciuu_agent = ReActAgent(
    name = "ciuu",
    description="use this agent to get information from the ciuu database",
    tools=query_engine_tools,
    llm=llm,
    system_prompt="""Eres un asistente experto en clasificación económica que ayuda a las personas a identificar el código CIIU (Clasificación Industrial Internacional Uniforme) que mejor se ajusta a la descripción de su actividad económica. Tu objetivo es analizar cuidadosamente la descripción que proporciona el usuario y encontrar el código CIIU más relevante y preciso.

                    Tienes acceso a una base de datos con códigos CIIU, sus nombres y descripciones. Cuando recibas una descripción de trabajo, debes:

                    1. Comprender el propósito principal de la actividad descrita.
                    2. Buscar el código CIIU que más se asemeje en términos de sector, producto, o servicio ofrecido.
                    3. Priorizar coincidencias conceptuales y semánticas, no solo palabras clave.
                    4. En caso de dudas, elige las 3 opciones mas específica y directa que represente la actividad.
                    5. Devuelve el resultado en el siguiente formato en json:

                    Código CIIU: [CÓDIGO]  
                    Nombre: [NOMBRE DE LA ACTIVIDAD]  
                    Si el usuario no proporciona suficiente información, responde solicitando una descripción más detallada de la actividad.

                    Responde siempre en español y con un lenguaje claro y formal.

                    no te olvides nunca de responder en el formato dado y el codigo tiene que ser el numero asociado al nombre de la actividad, no el nombre de la actividad solo.
                    Si no encuentras el código CIIU, responde con "No se encontró el código CIIU para la actividad proporcionada".

                   responde con un json en el siguiente formato
                    
                    
                """
)

ctx = Context(ciuu_agent)

In [76]:
handler = ciuu_agent.run("Soy profesor de primaria en un pueblo, dame mi codigo ciiu", ctx=ctx)

response = await handler

In [77]:
print(str(response))

El código CIIU que corresponde a tu actividad como profesor de primaria es el **8513: Educación básica primaria**.
